In [1]:
# Importing Libraries 
import numpy as np  # For all Maths
import pandas as pd # For Analysis
import matplotlib.pyplot as plt # For Visulisation
import seaborn as sns
%matplotlib inline

In [3]:
# Importing the Data 
data = pd.read_csv(r'C:\Users\Akhtar\Downloads\Compressed\training.1600000.processed.noemoticon.csv',encoding = "ISO-8859-1",header=None)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
0    1600000 non-null int64
1    1600000 non-null int64
2    1600000 non-null object
3    1600000 non-null object
4    1600000 non-null object
5    1600000 non-null object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [4]:
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [5]:
data.rename(columns={0:"Sentiment",1:'ID',2:'Date',3:'Flag',4:'User_Id',5:'Tweet'},inplace=True)

In [6]:
data.head()

,Sentiment,ID,Date,Flag,User_Id,Tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [7]:
data.Sentiment.value_counts()
# 4 is indicating a Positie Sentiment
# 0 is indicating a Negative Sentiment
# The data is very much balanced between the Labels since they are 50-50 %.

4    800000
0    800000
Name: Sentiment, dtype: int64

In [8]:
data.shape

(1600000, 6)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
Sentiment    1600000 non-null int64
ID           1600000 non-null int64
Date         1600000 non-null object
Flag         1600000 non-null object
User_Id      1600000 non-null object
Tweet        1600000 non-null object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [10]:
data.isnull().sum()
# There are no Missing Values in the Data Set.

Sentiment    0
ID           0
Date         0
Flag         0
User_Id      0
Tweet        0
dtype: int64

### Data Cleaning & Exploration

In [11]:
#import regex
import re

#start process_tweet
def processTweet(tweet): # Based on Domain Knowledge of Twitter here
    # process the tweets

    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub(r'@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub(r'[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', '\1', tweet)
    #trim
    tweet = tweet.strip('\'?,."')
    return tweet
#end

In [12]:
data.Tweet = data.Tweet.apply(lambda x :processTweet(x))

In [13]:
data.Tweet.head()

0    AT_USER URL - awww, that's a bummer. you shoul...
1    is upset that he can't update his facebook by ...
2    AT_USER i dived many times for the ball. manag...
3      my whole body feels itchy and like its on fire 
4    AT_USER no, it's not behaving at all. i'm mad....
Name: Tweet, dtype: object

In [28]:
#start replaceTwoOrMore
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)
#end

In [29]:
data.Tweet = data.Tweet.apply(lambda x : replaceTwoOrMore(x))

In [30]:
data.Tweet.head(10)

0    AT_USER URL - aww, that's a bummer. you should...
1    is upset that he can't update his facebook by ...
2    AT_USER i dived many times for the ball. manag...
3      my whole body feels itchy and like its on fire 
4    AT_USER no, it's not behaving at all. i'm mad....
5                          AT_USER not the whole crew 
6                                          need a hug 
7    AT_USER hey long time no see! yes.. rains a bi...
8                    AT_USER nope they didn't have it 
9                              AT_USER que me muera ? 
Name: Tweet, dtype: object

In [31]:
# Importing Libraries particularly used for Text Mining in Python
import nltk.classify.util
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

In [32]:
stopWords = (nltk.corpus.stopwords.words('english'))

In [33]:
stopWords.extend(['AT_USER','URL', 'lol' , 'aww' , 'bit',  'hey' ,'hi', 'hello', 'ok' , 'okay' ,'okk', 'ya', 'yaa' ,'so' ,'soo', 'awww'])
# Adding 'AT_USER' as they are those words starting with Hashtag which is very common for Twitter
# Adding URL to remove the URL's
# Adding extra words based on irrelevance of these words to determine the Sentiment.

In [34]:
stopWords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [35]:
#start getfeatureVector
def getFeatureVector(tweet,stopWords):
    featureVector = []
    #split tweet into words
    words = tweet.split()
    for w in words:
        #check if the word starts with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return featureVector
#end

In [36]:
data.Tweet = data.Tweet.apply(lambda x : getFeatureVector(x,stopWords))

In [37]:
data.Tweet.head(100)

0               [shoulda, got, david, carr, third, day]
1     [upset, update, facebook, texting, might, cry,...
2     [dived, many, times, managed, save, rest, go, ...
3               [whole, body, feels, itchy, like, fire]
4                                       [behaving, see]
5                                         [whole, crew]
6                                           [need, hug]
7                     [long, time, rains, fine, thanks]
8                                                [nope]
9                                          [que, muera]
10                      [spring, break, plain, snowing]
11                                               [ears]
12       [bear, watch, thought, ua, loss, embarrassing]
13                          [idk, never, talk, anymore]
14                           [really, zac, doucheclown]
15                             [wish, got, watch, miss]
16    [death, scene, hurt, severely, watch, film, wr...
17                                        [file,

In [38]:
data.Tweet.shape

(1600000,)

### Splitting the data into Train and Test

In [39]:
from sklearn.model_selection import  train_test_split

In [40]:
# Converted the data to 'text' since Count Vectorizer works only for Text not for Lists.
data.Tweet = [" ".join(tweets) for tweets in data['Tweet'].values]

In [41]:
data.Tweet.dtype

dtype('O')

In [42]:
# define X and y
X = data.Tweet
y = data.Sentiment

In [43]:
# split the new DataFrame into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1200000,)
(400000,)
(1200000,)
(400000,)


In [44]:
#FeatureEngineering
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, decomposition, ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import  textblob
from textblob import TextBlob
import nltk
from textblob import Word

### Defining CountVectorizer with Unigrams

In [45]:
X_train.head()

1389240          days make nooise
635813                     wanted
176401                   miss say
1282368            thanks mention
541863     ever get another turbo
Name: Tweet, dtype: object

In [46]:
#Train
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', 
                             ngram_range=(1, 1 ), 
                             min_df=5, #minimum document frequency
                             encoding = 'ISO-8859-1',
                             max_features=800)


In [47]:
xtrain_count = count_vect.fit_transform(X_train) # applying fit_transform on X_train

### Applying TF-IDF(Term Frequency-Inverse Document Frequency) on CountVectorized Data

In [48]:
from sklearn.feature_extraction.text import TfidfTransformer

In [49]:
# Applying tf-idf transformer on train data

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(xtrain_count) # xtrain_count is Count_Vectorizer applied on X_train

In [50]:
#Test
#count_vect = CountVectorizer()
xtest_count = count_vect.transform(X_test)

#tfidf_transformer = TfidfTransformer()
X_test_tfidf = tfidf_transformer.transform(xtest_count)

### Creating Bi-grams and Character level TF-IDF as selection parameters for Best Accuracy in Model Building

In [39]:
# ngram level tf-idf [ Fit and transform working is different here]

tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1, 2), max_features=800)
tfidf_vect_ngram.fit(data['Tweet'])   # Applying fit on target variable i.e data["Tweet"]
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(X_train)   # then , transform on X_train and X_test
xtest_tfidf_ngram =  tfidf_vect_ngram.transform(X_test)

In [40]:
# characters level tf-idf [Same as above] ,Here analyzer = "char"

tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(1,2), max_features=800)
tfidf_vect_ngram_chars.fit(data['Tweet'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_train) 
xtest_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_test)

### WORDCLOUD (MOST REPETITVE WORDS)

In [41]:
from wordcloud import WordCloud ,STOPWORDS

In [42]:
wordcloud = WordCloud(stopwords=[]).generate(' '.join(X_train.tolist()))

%matplotlib inline
fig = plt.figure(figsize=(200,100))
plt.imshow(wordcloud)

### Function defined for Model Building and finding Predictions for Test Data

In [43]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid,  valid_y, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

### Applying Naive Bayes (Best accuracy for Count Vectorized Tweets)

In [44]:
#Naive Bayes
# Naive Bayes on Count Vectors and TF-IDF
accuracy = train_model(naive_bayes.MultinomialNB(), X_train_tfidf, y_train, X_test_tfidf, y_test)

print("NB  for Count Vectors: ", accuracy)



# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, y_train, xtest_count, y_test)
print("NB  for WordLevel TF-IDF: ", accuracy)



# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, y_train, xtest_tfidf_ngram, y_test)
print("NB  for N-Gram Vectors: ", accuracy)



# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, y_train, xtest_tfidf_ngram_chars, y_test)
print("NB for  CharLevel Vectors: ", accuracy)

NB  for L1, Count Vectors:  0.70965
NB  for L1, WordLevel TF-IDF:  0.709625
NB  for L1, N-Gram Vectors:  0.70886
NB for L1, CharLevel Vectors:  0.6384675


### Applying Logitic Regression (Best accuracy for COunt Vectorized Tweets)

In [45]:
#Logistic Regression
# Logistic Regression on Count Vectors and TF-IDF
accuracy = train_model(LogisticRegression(), X_train_tfidf, y_train, X_test_tfidf, y_test)
print("LR  for Count Vectors: ", accuracy)



# Logistic Regression on Word Level TF IDF Vectors
accuracy = train_model(LogisticRegression(), xtrain_count, y_train, xtest_count, y_test)
print("LR  for WordLevel TF-IDF: ", accuracy)



# Logistic Regression on Ngram Level TF IDF Vectors
accuracy = train_model(LogisticRegression(), xtrain_tfidf_ngram, y_train, xtest_tfidf_ngram, y_test)
print("LR  for N-Gram Vectors: ", accuracy)



# Logistic Regression on Character Level TF IDF Vectors
accuracy = train_model(LogisticRegression(), xtrain_tfidf_ngram_chars, y_train, xtest_tfidf_ngram_chars, y_test)
print("LR for CharLevel Vectors: ", accuracy)

C:\Users\Akhtar\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LR  for L1, Count Vectors:  0.7162725
LR  for L1, WordLevel TF-IDF:  0.715905
LR  for L1, N-Gram Vectors:  0.715675
LR for L1, CharLevel Vectors:  0.66375


### We are getting an Accuracy of 70% which indicates the Model is Good.